# TD 5: Numerical integration of ODEs, and application to the Leaky Integrate-and-Fire neuron model
---

In this tutorial, we will first introduce methods for the numerical integration of differential equations. We will then use those methods to implement a simulation of an Integrate-and-Fire neuron.

## A short introduction to the numerical integration of differential equations

The time evolution of physical systems (planets & membrane voltages) is often described by differential equations, i.e., some equation of the type

$$ \frac{dx}{dt} = f(x, t) $$

When those equations are simple (when $f$ is a simple function), we can solve such equations analytically. An example would be $\dot x = -x/\tau$.

However, it may come in handy to be able to solve such equations numerically, either to check an analytical result or because we haven't been able to fine one (it might not even exist in closed form).



In [ ]:
# some imports for the numerics to follow, you'll be familiar with these
%matplotlib notebook
from pylab import *

In [ ]:
# Decay dynamics

tau = 7.    # decay time constant, in ms
V0 = 20.    # initial voltage, in mV
EL = -50.   # resting potential, in mV

t = arange(0, 100, 10)

# We know the ODE that governs the
# dynamics of the potential!
def dvdt(v):
    return (EL+I-V)/tau

# For this simple case, we already know
# the solution of the ODE:
def v_theory(t):
    return (V0-EL)*exp(-t/tau) + EL

plot(t, v_theory(t))
xlabel('time (ms)')
ylabel('voltage (mV)')

Question: can we compute the time course $V(t)$ of the potential by using the ODE?

#### Euler method

A very simple and straightforward method to numerically solve differential equations is the *Euler method*, which relies on the discretization in time of the dynamics and the approximation of the time derivative by a finite difference between subsequent time points:

$$\frac{dx}{dt}\approx\frac{\Delta x}{\Delta t} \ \ \Rightarrow \ \ \Delta x \approx \Delta t \frac{dx}{dt}=\Delta t f(x,t)$$

That is, for $x(t+\Delta t) = x(t) + \Delta x$ we obtain:

$$ x(t+\Delta t) = x(t) + \Delta t f(x,t)$$

In [ ]:
# Can you translate the Euler formula into an algorithm?

# We should define a time step and an integration time:
dt = 0.1
T = 100.
nt = int(T/dt)

# An empty array to be filled with the computed values
# of the voltage:
v_euler = zeros(nt)

# Initial condition
v_euler[0] = V0

# ...

In [ ]:
# Plot here the theoretical result 
# together with your numerical solution.

When using the Euler integration method, **the result crucially depends on the time step** used! 

*Question:* How do we know that the result is correct? 

*Answer:* Repeat integration with finer resolution (half the time step) and check for convergence!

In [ ]:
# Repeat the above for different time steps and compare.

#### Other methods: using the `scipy` ODE solver

The undeniable advantage of the Euler method is it's simplicity. In many circumstances, one can use much more powerful algorithms; powerful in the sense of better convergence to the true result with respect to the time step used, robustness, etc. However, in almost all practical situations, the easiest option is to use built-in ODE solvers. One such solver is part of the `scipy` python module: `scipy.optimize.odeint` 


In [ ]:
# We can use a numerical solver that is already 
# programmed for us!
from scipy.integrate import odeint

In [ ]:
# check out the documentation
odeint?

In [ ]:
# Can you use odeint to obtain another numerical solution
# for v(t)? 

In [ ]:
# Plot a comparison of the theoretical and the two
# numerical solutions here.

**NOTE:** `scipy.odeint` requires the derivative function to take time as second argument!

## Simulation of a Leaky Integrate-and-Fire neuron

We are now equipped to simulate a LIF neuron for arbitrary input current. Think about which method you would choose to solve the ODE for the membrane potential. Do you have an idea how to implement the threshold condition and the reset?

In [ ]:
# Try to solve the equation for the LIF.
# Start with a constant input current below the firing threshold.

dt = 0.01 # time step in ms
T = 100.0 # duration in ms
t = arange(0,T,dt)

# ...

Of great interest in neuroscience is generally the timing of spikes of individual neurons. The attractivity of Integrate-and-Fire neurons is their possibility to generate such spiketimes in well-controlled settings.

In [ ]:
# How does the membrane voltage evolve with a constant input current 
# above the firing threshold?

# When the neuron spikes, add the spiketime to a list:
spiketimes = []
# --> whenever there's a spike at time 'time', do 
spiketimes.append(time)


In [ ]:
# Plot the membrane potential v(t) together with the spikes
dummytimes = [3, 23, 49, 52, 78, 90]
plot(t, sin(0.1*t), label='demo voltage')
for time in dummytimes:
    axvline(time, color='k')
plot([],[],'k',label='spikes')
legend()

We can use our numerical solution of the LIF to check the theoretical prediction of the firing rate, i.e.,

$$f(I) = \frac{1}{\tau_m} \left(\log\frac{E_L+I/g_L-V_{\rm reset}}{E_L+I/g_L-V_{\rm threshold}}\right)^{-1}$$

In [ ]:
# Can you calculate the firing rate (the inverse 
# interspike interval, or ISI) for different values
# of the input current?

### Simulation of a noisy LIF!

Instead of a deterministic, or even constant input current $I(t)$, we now consider a noisy input that represents the combined effect of many presynaptic neurons:

$$I(t) = I_0 + \sigma \xi(t),$$

where $I_0$ is the (constant) mean input, $\sigma$ is a parameter that represents the noise strength, and $\xi(t)$ is a Gaussian stochastic process (normal-distributed) with correlation function $\langle\xi(t)\xi(t')\rangle=\delta(t-t')$. 

In discretized form, the resulting _**stochastic differential equation**_ becomes

$$v_{i+1} = v_i + \Delta t \left[\left(\frac{dv}{dt}\right)_{\rm det}(v_i, t_i) + \sigma\sqrt{\frac{\tau_m}{\Delta t}} \xi_i\right],$$

where $\xi_i$ is now a random number drawn from a normal distribution with zero mean and normalized standard deviation.

In [ ]:
# We get a single random number using
xi = randn()
print(xi)

# We can check that the histogram of many of 
# of these gives a normal (Gauss) distribution:
hist(randn(1000000), bins=100, density=True)
x = linspace(-4,4,201)
plot(x, exp(-x**2/2)/sqrt(2*pi), color='r',linewidth=2)

In [ ]:
# Can you solve the stochastic ODE numerically?
sigma = 2.0

# ...